# **HOW TO TRAIN NEW MODEL (En to Vi)**



Phiên bản V1 nhằm chạy thử model TED tiếng việt với các đặc điểm:

1 - Chạy duy nhất 1 epoch

2 - Thực hiện trên máy local


## Data Pre-processing

In [ ]:
# # Download and prepare the data
!bash clc_fairseq/examples/translation/prepare-en-vi-iwslt.sh


Cloning Moses github repository (for tokenization scripts)...
Cloning into 'mosesdecoder'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 147592 (delta 5), reused 11 (delta 3), pack-reused 147572
Receiving objects: 100% (147592/147592), 129.76 MiB | 7.84 MiB/s, done.
Resolving deltas: 100% (114030/114030), done.
Checking out files: 100% (3470/3470), done.
Cloning Subword NMT repository (for BPE pre-processing)...
Cloning into 'subword-nmt'...
remote: Enumerating objects: 576, done.
remote: Total 576 (delta 0), reused 0 (delta 0), pack-reused 576
Receiving objects: 100% (576/576), 233.12 KiB | 3.33 MiB/s, done.
Resolving deltas: 100% (349/349), done.
--2020-11-21 06:37:56--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sen

# Training

In [ ]:
!CUDA_VISIBLE_DEVICES=0 fairseq-train "iwslt-data-bin/tokenized.en-vi" \
    --arch transformer --share-decoder-input-output-embed \
    --optimizer adam --adam-betas '(0.9, 0.98)' --clip-norm 0.0 \
    --lr 5e-4 --lr-scheduler inverse_sqrt --warmup-updates 400 \
    --dropout 0.3 --weight-decay 0.0001 \
    --criterion label_smoothed_cross_entropy --label-smoothing 0.1 \
    --max-tokens 4096 \
    --eval-bleu \
    --eval-bleu-args '{"beam": 5, "max_len_a": 1.2, "max_len_b": 10}' \
    --eval-bleu-detok moses \
    --eval-bleu-remove-bpe \
    --eval-bleu-print-samples \
    --best-checkpoint-metric bleu --maximize-best-checkpoint-metric \
    --save-dir "iwslt-checkpoints/fconv_wmt_en_vi" \
    --max-epoch 1



/usr/local/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0
2020-11-23 03:28:56 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': None, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min

In [1]:
!pwd

/home/chinh/clc_fairseq


In [2]:
# Evaluate
!mkdir -p iwslt-data-bin/fconv_wmt_en_vi
!fairseq-generate \
    iwslt-data-bin/tokenized.en-vi/ \
    --path iwslt-checkpoints/fconv_wmt_en_vi/checkpoint_best.pt \
    --beam 5 --remove-bpe | tee iwslt-data-bin/fconv_wmt_en_vi/gen.out

2020-11-25 21:05:40 | INFO | fairseq_cli.generate | {'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'tensorboard_logdir': None, 'wandb_project': None, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1, 'quantization_config_path': None, 'profile': False}, 'common_eval': {'_name': None, 'path': 'iwslt-checkpoints/fconv_wmt_en_vi/checkpoint_best.pt', 'post_process': '@@ ', 'quiet': False, 'model_overrides': '{}', 'results_path': None}, 'distributed_training': {'_name': None, 'distributed_world_size': 1, 'distributed_rank': 0, 'distributed_backend': 'nccl', 'distributed_init_method': None, 'distributed_

  0%|                                                     | 0/7 [00:00<?, ?it/s]S-1046	we were hired to replace this building , which burned down .
T-1046	chúng tôi được thuê để thay thế toà nhà này , nó bị cháy rụi .
H-1046	-2.0460569858551025	chúng ta có thể tạo ra một cách khác nhau .
D-1046	-2.0460569858551025	chúng ta có thể tạo ra một cách khác nhau .
P-1046	-0.8969 -1.4577 -2.4504 -0.9856 -4.6484 -0.7458 -2.8416 -3.9117 -3.3326 -1.3454 -1.6724 -0.2642
S-529	how do we get to augmented reality , a.r. ?
T-529	làm sao có được ar ?
H-529	-1.6201436519622803	chúng ta có thể làm sao ?
D-529	-1.6201436519622803	chúng ta có thể làm sao ?
P-529	-2.9568 -0.3486 -2.1550 -0.4343 -2.7706 -2.2970 -0.9863 -1.0124
S-896	but , what &amp; apos ; s the problem with all of this ?
T-896	nhưng vấn đề với tất cả là gì ?
H-896	-1.767630934715271	nhưng nếu bạn có thể làm sao ?
D-896	-1.767630934715271	nhưng nếu bạn có thể làm sao ?
P-896	-2.3533 -2.6110 -0.4677 -2.2520 -0.3515 -3.4509 -2.2761 -1.8805 -0.

P-447	-3.0408 -1.0516 -2.8776 -2.6525 -2.4448 -1.5602 -0.2815 -0.0623 -2.4102 -4.4209 -1.9135 -0.3444 -0.0654 -2.7122 -3.1510 -4.2269 -2.6670 -3.2581 -1.3620 -1.4207 -0.3582 -0.0686 -0.1255
S-49	now , there &amp; apos ; s a profound irony here .
T-49	đến đây ta gặp một sự trớ trêu .
H-49	-1.7126013040542603	bạn có thể thấy nó .
D-49	-1.7126013040542603	bạn có thể thấy nó .
P-49	-4.1725 -1.6475 -0.4436 -1.5672 -2.2651 -1.6787 -0.2136
S-584	we know what this technology is worth to sports leagues .
T-584	công nghệ này đáng giá với các giải đấu ,
H-584	-1.6452046632766724	chúng ta có thể làm việc với chúng ta .
D-584	-1.6452046632766724	chúng ta có thể làm việc với chúng ta .
P-584	-1.1960 -0.7489 -2.4213 -0.6027 -3.1465 -2.6544 -2.5420 -3.5287 -0.4885 -0.6221 -0.1463
S-562	well , imagine you &amp; apos ; re the quarterback .
T-562	hình dung , bạn là cầu thủ chuyền bóng .
H-562	-1.5270943641662598	bạn có thể thấy nó .
D-562	-1.5270943641662598	bạn có thể thấy nó .
P-562	-3.1845 -1.5351 -0.

H-277	-2.0133256912231445	đó là một điều mà chúng ta có thể làm .
D-277	-2.0133256912231445	đó là một điều mà chúng ta có thể làm .
P-277	-3.3062 -0.0828 -1.9460 -4.1293 -2.2849 -2.1329 -0.7303 -1.8163 -0.7206 -3.6764 -3.2527 -0.0814
S-356	i could no longer live my truth for and by myself .
T-356	tôi không thể chỉ sống thật cho riêng mình .
H-356	-1.660477876663208	và tôi muốn nói với các bạn .
D-356	-1.660477876663208	và tôi muốn nói với các bạn .
P-356	-1.4103 -0.3297 -3.5337 -1.8664 -2.0205 -3.0137 -0.1027 -2.3771 -0.2902
S-747	it &amp; apos ; s not a science of explanations .
T-747	nó không phải là khoa học giải thích .
H-747	-1.6461503505706787	bạn có thể thấy nó .
D-747	-1.6461503505706787	bạn có thể thấy nó .
P-747	-3.8776 -1.3622 -0.3752 -1.8257 -2.5043 -1.4108 -0.1673
S-723	there &amp; apos ; s nothing we know about more directly .
T-723	chúng ta không biết gì khác trực tiếp hơn
H-723	-1.5655453205108643	nhưng chúng ta có thể làm thế nào .
D-723	-1.5655453205108643	nhưng chúng

P-269	-4.1280 -3.2050 -0.0831 -1.5850 -1.4628 -1.3554 -1.3554 -1.3914 -1.3924 -1.3324 -1.2436 -1.1700 -1.1294 -1.1305 -1.1734 -1.2293 -1.2564 -1.2547 -1.2466 -1.2498 -1.2650 -1.2926 -1.3113 -1.3108 -1.3032 -1.3033 -1.3150 -1.3247 -1.3191 -4.1487 -0.2513 -0.1250 -0.9873 -1.1786 -1.2036 -1.1868 -1.1476 -1.1168 -1.1145 -1.1436 -1.1812 -1.1908 -1.1614 -1.1140 -1.0742 -1.0606 -1.0791 -1.1122 -1.1263 -1.1220 -1.1296 -1.1689 -1.2452 -1.3331 -1.3734 -1.3426 -1.2791 -1.2359 -1.2283 -1.2390 -1.2396 -1.2141 -1.1729 -1.1470 -1.1608 -1.2091 -1.2638 -1.2817 -1.2543 -1.2047 -1.1703 -4.2906 -0.2312 -0.1403 -3.1240 -0.2244 -0.1207 -1.0285 -1.0370 -1.0685 -1.1236 -1.1794 -1.2049 -1.1964 -1.1774 -1.1684 -1.1800 -1.2207 -1.2725 -1.3144 -1.3218 -1.2853 -1.2167 -1.1532 -1.1133 -1.0996 -1.1094 -1.1211 -1.1200 -1.1031 -1.0799 -1.0667 -1.0685 -1.0769 -1.0738 -1.0524 -1.0216 -1.0027 -1.0044 -1.0303 -1.0552 -1.0640 -1.0498 -1.0243 -1.0049 -1.0129 -1.0473 -1.0921 -1.1247 -1.1289 -1.1025 -1.0696 -1.0493 -1.0534 -1

...

T-160	ngày nay , người ta tranh cãi về điều này và có nhiều cuộc tranh luận về cách hoạt động của tâm trí , nhưng với tôi , đơn giản là : hình ảnh trong tâm trí , là trung tâm của suy nghĩ đổi mới và sáng tạo .
H-160	-2.2207934856414795	và sau đó , tôi đã nói với các bạn , và chúng ta có thể làm việc với các bạn .
D-160	-2.2207934856414795	và sau đó , tôi đã nói với các bạn , và chúng ta có thể làm việc với các bạn .
P-160	-1.2330 -4.6015 -0.7333 -1.1834 -1.2560 -2.9889 -3.1182 -2.1387 -2.5209 -0.5260 -1.8815 -2.4658 -4.3824 -1.0037 -2.8801 -1.1416 -4.0676 -2.2926 -2.5016 -3.2123 -1.1350 -3.5486 -0.2655
S-587	what is this technology worth to a commander hadfield or a neil degrasse tyson trying to inspire a generation of children to think more about space and science instead of quarterly reports and kardashians ?
T-587	công nghệ đáng giá thế nào với một tư lệnh hadfiel hay một neil <<unk>> sse tyson khi truyền cảm hứng cho thế hệ trẻ suy nghĩ nhiều hơn về không gian và khoa học thay vì 

S-759	you could apply that to the human brain in explaining some of the behaviors and the functions of the human brain as emergent phenomena : how we walk , how we talk , how we play chess , all these questions about behavior .
T-759	chúng ta có thể áp dụng cho não người trong việc giải thích một số hành vi và chức năng của nó như là hiện tượng tự phát : cách chúng ta đi lại , nói chuyện , cách chúng ta chơi cờ , và mọi câu hỏi khác về hành vi .
H-759	-2.1773300170898438	và chúng ta có thể nói về những gì chúng ta có thể nói về những gì chúng ta có thể làm cho chúng ta .
D-759	-2.1773300170898438	và chúng ta có thể nói về những gì chúng ta có thể nói về những gì chúng ta có thể làm cho chúng ta .
P-759	-1.3869 -2.9465 -0.9149 -2.4148 -0.6049 -3.3345 -1.9530 -3.0035 -2.8415 -2.2558 -0.6107 -2.4340 -0.5778 -4.2690 -1.7748 -2.8435 -3.0894 -2.6845 -0.6241 -2.6088 -0.5577 -3.0178 -3.5242 -3.4744 -0.6193 -3.9417 -0.4801
S-60	your closet may be telling someone you love her for the first time 

D-583	-2.1040947437286377	và tôi nghĩ rằng chúng ta có thể làm việc với các bạn .
P-583	-2.3110 -0.8935 -2.5283 -1.2362 -3.0855 -1.0940 -2.4247 -0.9397 -3.6086 -2.3757 -2.3345 -3.6207 -1.5894 -3.1796 -0.3401
S-194	but yet most of the recent advances in resolution have actually come from ingeniously clever encoding and decoding solutions in the f.m. radio frequency transmitters and receivers in the mri systems .
T-194	nhưng những tiến bộ gần đây về độ phân giải lại đến từ những giải pháp thông minh trong mã hóa và giải mã trong dẫn truyền tần số sóng fm và đầu nhận của hệ thống mri .
H-194	-2.4145617485046387	và sau đó , chúng tôi đã có một phần trăm năm trước .
D-194	-2.4145617485046387	và sau đó , chúng tôi đã có một phần trăm năm trước .
P-194	-1.3454 -4.7044 -0.4931 -1.5047 -3.0367 -0.9192 -2.4296 -3.6587 -2.0490 -4.2114 -1.3734 -3.9013 -3.0717 -3.3240 -0.1959
S-979	and the people that build things -- developers and governments -- they &amp; apos ; re naturally afraid of innovation 

T-1049	chúng ta đã đề xuất một toà nhà rất táo bạo , rất khác biệt với tất cả những khuôn mẫu mà cộng đồng vốn quen thuộc , và chúng tôi lo lắng và khách hàng của chúng tôi cũng cảm thấy lo lắng và cộng đồng cũng lo lắng , nên chúng tôi đã tạo ra một loạt những bản dựng hình ảnh như thật và chúng tôi đăng lên facebook và chúng tôi đăng lên instagram , và chúng tôi để mọi người bắt đầu làm điều mà họ làm : chia sẻ nó , bình luận , thích nó , ghét nó .
H-1049	-2.2822275161743164	và chúng tôi đã nói rằng , và chúng ta có thể làm việc với những người khác nhau , và chúng ta có thể làm việc , và chúng ta có thể làm việc với nhau , và chúng ta , và chúng ta có thể làm cho chúng ta .
D-1049	-2.2822275161743164	và chúng tôi đã nói rằng , và chúng ta có thể làm việc với những người khác nhau , và chúng ta có thể làm việc , và chúng ta có thể làm việc với nhau , và chúng ta , và chúng ta có thể làm cho chúng ta .
P-1049	-0.7980 -2.0998 -0.5066 -2.4771 -3.8232 -2.1291 -2.6045 -1.9416 -2.3026 -1.4

D-827	-2.204303026199341	và tôi nghĩ rằng , nếu bạn có thể nhìn thấy nó , bạn có thể có thể thấy nó .
P-827	-1.5088 -1.4815 -3.4000 -1.6846 -2.2636 -2.9907 -0.6710 -2.5173 -0.6547 -4.2902 -1.2925 -3.2755 -1.9979 -3.2181 -2.1980 -0.6570 -4.3713 -0.7512 -2.9026 -2.9511 -3.1267 -0.2903
S-7	pretend that you &amp; apos ; re an alien race that doesn &amp; apos ; t know anything about earth biology or earth neuroscience or earth intelligence , but you have amazing telescopes and you &amp; apos ; re able to watch the earth , and you have amazingly long lives , so you &amp; apos ; re able to watch the earth over millions , even billions of years .
T-7	giả sử quý vị thuộc một chủng người ngoài hành tinh không biết gì về sinh học trái đất , về khoa thần kinh , hay trí thông minh trái đât , nhưng có chiếc kính viễn vọng rất tuyệt giúp quan sát trái đất , và đời quý vị rất dài nên có thể quan sát hàng triệu , thậm chí hàng tỉ năm .
H-7	-2.301861047744751	bạn biết đấy , nếu bạn có thể nhìn thấy , bạ

Compute BLEU score


In [4]:
!grep ^H iwslt-data-bin/fconv_wmt_en_vi/gen.out | cut -f3- > iwslt-data-bin/fconv_wmt_en_vi/gen.out.sys
!grep ^T iwslt-data-bin/fconv_wmt_en_vi/gen.out | cut -f2- > iwslt-data-bin/fconv_wmt_en_vi/gen.out.ref
!fairseq-score --sys iwslt-data-bin/fconv_wmt_en_vi/gen.out.sys --ref iwslt-data-bin/fconv_wmt_en_vi/gen.out.ref


Namespace(ignore_case=False, order=4, ref='iwslt-data-bin/fconv_wmt_en_vi/gen.out.ref', sacrebleu=False, sentence_bleu=False, sys='iwslt-data-bin/fconv_wmt_en_vi/gen.out.sys')
BLEU4 = 2.18, 20.5/4.4/1.5/0.3 (BP=0.841, ratio=0.853, syslen=22656, reflen=26566)


BLEU4 = 2.18 quá thấp, cần xem lại 